In [2]:
import sys
import os
import numpy as np

sys.path.append(os.path.abspath("../../mathematical_models"))
from mathematical_models.s_on_f import ScalarOnFunctionModel
from mathematical_models.f_on_f import FunctionOnFunctionModel
from mathematical_models.s_on_s import ScalarOnScalarModel

sys.path.append(os.path.abspath("../../optimizers"))
from optimizers.nbdo import NBDO

sys.path.append(os.path.abspath("../../utilities"))
from utilities.plotting.plot_fun import subplot_results

### Settings

In [3]:
N = 24 # runs
Kx = [[3]]
epochs = 100
order = 1
s_on_s_model = ScalarOnScalarModel(Kx=Kx[0], order=order)

In [4]:
optimizer_s_on_s = NBDO(model=s_on_s_model, latent_dim=4)
optimizer_s_on_s.compute_train_set(num_designs=1_000, runs=N)

In [5]:
print(optimizer_s_on_s.train_set[0].shape)

(96,)


In [6]:
history = optimizer_s_on_s.fit(epochs=epochs)

Epoch 1/100


2024-09-23 14:49:07.338334: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


ValueError: in user code:

    File "/Users/thodorisladas/miniforge3/envs/optidex/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/Users/thodorisladas/Documents/code/system_optidex/optimizers/nbdo.py", line 148, in custom_loss  *
        objective_value = self.model.compute_objective_tf(y_pred, m, n)
    File "/Users/thodorisladas/Documents/code/system_optidex/mathematical_models/s_on_s.py", line 55, in compute_objective_tf  *
        X = tf.reshape(X, (-1, m, n))

    ValueError: Dimension size must be evenly divisible by 72 but is 3072 for '{{node custom_loss/Reshape}} = Reshape[T=DT_FLOAT, Tshape=DT_INT32](autoencoder/decoder/dense_8/Tanh, custom_loss/Reshape/shape)' with input shapes: [32,96], [3] and with input tensors computed as partial shapes: input[1] = [?,24,3].


In [4]:
best_cr, best_des = optimizer_s_on_s.optimize()

Iteration No: 1 started. Evaluating function at random point.
1/1 [==============================] - 0s 38ms/step
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0552
Function value obtained: 0.2602
Current minimum: 0.2602
Iteration No: 2 started. Evaluating function at random point.
1/1 [==============================] - 0s 13ms/step
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0293
Function value obtained: 0.3039
Current minimum: 0.2602
Iteration No: 3 started. Evaluating function at random point.
1/1 [==============================] - 0s 9ms/step
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0250
Function value obtained: 0.2671
Current minimum: 0.2602
Iteration No: 4 started. Evaluating function at random point.
1/1 [==============================] - 0s 8ms/step
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0223
Function value obtained: 0.2587
Current minimum: 0.2587
Iteration No: 5 starte

In [5]:
print(best_des)
# 0.2551

[[ 0.99998975 -0.99999106  0.99997365 -0.9999347   0.99999326]
 [-0.9999893   0.99999976  0.99999243 -0.9999043   0.9999906 ]
 [ 0.99999624  0.99999976 -0.99999774  0.9999989   0.9999988 ]
 [-0.99999064  0.99999875 -0.99999183  0.9999917   0.99999934]
 [-0.99999493  0.9999972  -0.9999889   0.99999213 -0.99999505]
 [ 0.99999803 -0.9999967  -0.9999967  -0.99998945  0.9999989 ]
 [-0.999997    0.99999934  0.9999959   0.99999535  0.99999815]
 [ 0.9999981  -0.9999991  -0.9999998   0.99999917  0.9999971 ]
 [-0.99999964 -0.9999949   0.99999636  0.99999344  0.9999975 ]
 [-0.9999957  -0.9999994  -0.9999914   0.9999985  -0.99999803]
 [-0.99999404 -0.9999942   0.9999914   0.9999935   0.9999932 ]
 [ 0.9999964  -0.9999951  -0.99999475 -0.99999505  0.99998486]
 [ 0.9999982   0.99999744  0.9999958   0.9999974  -0.99999714]
 [-0.99998665  0.99999845 -0.99998885  0.99999255 -0.9999971 ]
 [ 0.9999993   0.99999964  0.9999988  -0.99999654 -0.99999976]
 [-0.9999966  -0.99999577  0.9999798  -0.9999994   0.99

In [6]:
best_cr

0.2586192493825103

In [7]:
import tensorflow as tf
random_tensor = tf.random.uniform(shape=(32, 120), minval=-1, maxval=1)

In [8]:
random_tensor_np = random_tensor.numpy()

In [9]:
print(random_tensor_np)

[[ 0.47360587  0.42970014  0.9150071  ... -0.08434439 -0.09980631
  -0.53753185]
 [ 0.06475973  0.15494227  0.7360747  ...  0.29003072 -0.8676915
  -0.44201994]
 [-0.15821862  0.606426    0.8916526  ...  0.01027966 -0.2113626
   0.34109664]
 ...
 [ 0.12963247 -0.29322696 -0.29689527 ... -0.23259735  0.8437507
  -0.83500886]
 [-0.33764434 -0.20710707  0.5258908  ... -0.35601425  0.36251664
   0.28831482]
 [ 0.99417424 -0.27418137 -0.61483335 ...  0.10298729  0.64278936
   0.20882654]]
